In [2]:
import importlib
from bs4 import BeautifulSoup
import json
import os
import re
import sys
import time
import traceback
from collections import namedtuple
from datetime import datetime
from io import StringIO, BytesIO
from xml.etree import ElementTree as ET
from urllib.robotparser import RobotFileParser
from io import StringIO

import numpy as np
import pandas as pd

In [5]:
def ab_test_helper(click_through=[], best=0):
    importlib.reload(main_mod)
    points = 0

    visits = 20  # how many times should we hit home page?
    learn = 10  # how many times does it try both before deciding?
    html = []  # HTML loaded from page for each visit

    for i in range(visits):
        status, headers, body = app_req("/")
        page = BeautifulSoup(body, "lxml")
        links = page.find_all("a", href=re.compile("donate.html\S*"))
        links = [link["href"] for link in links]
        if len(links) != 1:
            print("expected exactly one link to donate, but found", links)
            return 0
        if status != "200 OK":
            print("could not visit /")
            return 0

        html.append(body) #

        if i in click_through:
            status, headers, body = app_req(links[0])
            print(headers)
            if status != "200 OK":
                print("could not visit " + links[0])
                return 0

    def _transform(html: str):
        # regex didn't work with one of the submissions that had to be reran
        return html.replace('donate.html?from=A', '').replace('donate.html?from=B', '') 

    # phase 1: alternate
    for i in range(1, learn):
        # breakpoint()
        if _transform(html[i]) == _transform(html[i - 1]):
            if html[i] == html[i - 1]:
                print("(a) did not alternate html in first %d visits" % learn)
                return points
            else:

                global only_varied_query_str
                only_varied_query_str = True
        if i > 1 and html[i] != html[i - 2]:
            print("(b) did not alternate html in first %d visits" % learn)
            return points

    if only_varied_query_str:
        print("alternated between versions, but they only differ at the query string.")

    points += 1

    # phase 2: same
    for i in range(learn + 1, visits):
        if html[i] != html[i - 1]:
            print("did not consistently show same page after first %d visits" % learn)
            return points
    points += 2
    
    print(f"Final best version chosen: {html[best]}") # debugger
    
    # did they choose the best for phase 2?
    if html[learn] != html[best]:
        print("did not choose the best version")
    else:
        points += 2
    
    return points

@test(points=25)
def ab_test():
    points = 0
    points += ab_test_helper(click_through=[0], best=0)
    points += ab_test_helper(click_through=[1], best=1)
    points += ab_test_helper(click_through=[0, 2, 4, 6, 8, 3, 5, 7, 9], best=0)
    points += ab_test_helper(click_through=[2, 4, 6, 8, 1, 3, 5, 7, 9], best=1)
    points += ab_test_helper(click_through=[2, 4, 6, 8, 5, 7, 9], best=0)

    if only_varied_query_str:
        points -= 2
    return points

NameError: name 'test' is not defined

In [3]:
def main():
    # import main.py (or other, if specified)
    mod_name = "main"
    if len(sys.argv) > 2:
        print("Usage: python3 test.py [mod_name]")
        sys.exit(1)
    elif len(sys.argv) == 2:
        mod_name = sys.argv[1]

    run_all_tests(mod_name)


if __name__ == "__main__":
    main()

Usage: python3 test.py [mod_name]


SystemExit: 1

/home/chenchenmengmeng/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
1%2

1